#A Tale of a Reduction

> Objectives:
> * Compare operations taking place in different data containers
> * Compare sizes for these data containers
> * Help deciding when it is best to use a container or another

Let's suppose that we are going to exercise reductions a lot and we want to choose the best container for doing this.  First, let's start by activating our MemWatcher agent:

In [1]:
from ipython_memwatcher import MemWatcher
mw = MemWatcher()
mw.start_watching_memory()

In [1] used 0.031 MiB RAM in 0.001s, peaked 0.000 MiB above current, total RAM usage 30.961 MiB


and choose a different container for the data that we want to reduce, starting with a list:

##Regular lists

In [2]:
a = [i for i in range(1000*1000)]

In [2] used 31.250 MiB RAM in 0.109s, peaked 0.000 MiB above current, total RAM usage 62.211 MiB


Now, proceed with a simple reduction (sum):

In [3]:
%timeit sum(a)

The slowest run took 4.74 times longer than the fastest. This could mean that an intermediate result is being cached 
100 loops, best of 3: 6.95 ms per loop
In [3] used 0.023 MiB RAM in 2.993s, peaked 0.000 MiB above current, total RAM usage 62.234 MiB


which, in MIPS (Mega-Instructions-Per-Second) is:

In [4]:
print("MIPS:", round(1e6 / 7.0e-3 / 1e6, 3))

('MIPS:', 142.857)
In [4] used 0.000 MiB RAM in 0.002s, peaked 0.000 MiB above current, total RAM usage 62.234 MiB


Ok, so that seems fast, but we don't have other references to compare with.  In addition, a list is not best kind of container in terms of space consumption.  So let's try now a container that seems quite optimal in terms of memory savings.

In [5]:
# Create an array of 'l'ong integers (8 bytes on 32-bit platforms)
import array
arr = array.array('l', a)

In [5] used 7.387 MiB RAM in 0.095s, peaked 0.000 MiB above current, total RAM usage 69.621 MiB


7.7 MB vs 31 MB seems like a good deal.  In fact, the array module seems to provide optimal containers from a memory consumption point of view:

In [6]:
# Size per element:
(mw.memory_delta * 2**20) / 1e6

7.745536

In [6] used 0.055 MiB RAM in 0.019s, peaked 0.000 MiB above current, total RAM usage 69.676 MiB


But how it performs during reductions?

In [7]:
%timeit sum(arr)

100 loops, best of 3: 11.6 ms per loop
In [7] used 0.043 MiB RAM in 4.891s, peaked 0.000 MiB above current, total RAM usage 69.719 MiB


In [8]:
print("MIPS:", round(1e6 / 11.5e-3 / 1e6, 3))

('MIPS:', 86.957)
In [8] used 0.012 MiB RAM in 0.002s, peaked 0.000 MiB above current, total RAM usage 69.730 MiB


Well, that's a bit disappointing, as the array object performs a 60% slower than a regular array.  Not sure about the resons, but probably the array module is not getting too much attention performance-wise mainly because the NumPy existance.  Speaking of NumPy: here we go!

##NumPy

In [9]:
import numpy as np
na = np.array(a, dtype=np.int64)

In [9] used 22.125 MiB RAM in 0.177s, peaked 0.000 MiB above current, total RAM usage 91.855 MiB


In [10]:
%timeit sum(na)

10 loops, best of 3: 84.8 ms per loop
In [10] used 0.094 MiB RAM in 3.584s, peaked 0.000 MiB above current, total RAM usage 91.949 MiB


In [11]:
print("MIPS:", round(1e6 / 83.7e-3 / 1e6, 3))

('MIPS:', 11.947)
In [11] used 0.000 MiB RAM in 0.002s, peaked 0.000 MiB above current, total RAM usage 91.949 MiB


Oops, this is more than 7x slower than the `array` module.  Why so?

**Answer:** NumPy has a lot of overhead in producing a Python integer for every element in the array.

*Hint:* Use internal methods (ufuncs) when possible.

In [12]:
%timeit na.sum()

1000 loops, best of 3: 552 µs per loop
In [12] used 0.070 MiB RAM in 2.379s, peaked 0.000 MiB above current, total RAM usage 92.020 MiB


In [13]:
print("MIPS:", round(1e6 / 558e-6 / 1e6, 3))

('MIPS:', 1792.115)
In [13] used 0.000 MiB RAM in 0.003s, peaked 0.000 MiB above current, total RAM usage 92.020 MiB


This is more than 100x the speed of sum() on a Python list and it is also pretty optimal in terms of both execution time and space consumed.  

But let us suppose that we have really big data to process in our laptop and want to see if we can store our data in less space.  Enter compression:

##bcolz

In [14]:
import bcolz

In [14] used 22.938 MiB RAM in 0.372s, peaked 0.000 MiB above current, total RAM usage 114.957 MiB


In [15]:
ca = bcolz.carray(na)

In [15] used 8.215 MiB RAM in 0.016s, peaked 0.000 MiB above current, total RAM usage 123.172 MiB


Why so much memory consumption?  This is an artifact of the OS memory subsystem and is probably OS dependent.  Let's try again and create a new carray:

In [16]:
ca2 = bcolz.carray(na)

In [16] used 0.273 MiB RAM in 0.011s, peaked 0.000 MiB above current, total RAM usage 123.445 MiB


In [17]:
print("mem_used:", mw.measurements.memory_delta)

('mem_used:', 0.2734375)
In [17] used 0.008 MiB RAM in 0.003s, peaked 0.000 MiB above current, total RAM usage 123.453 MiB


Ok, this time the amount of memory used seems much lower.  Let's see how much memory the container thinks it has:

In [23]:
ca

carray((1000000,), int64)
  nbytes: 7.63 MB; cbytes: 394.28 KB; ratio: 19.81
  cparams := cparams(clevel=5, shuffle=True, cname='blosclz')
[     0      1      2 ..., 999997 999998 999999]

In [23] used 0.027 MiB RAM in 0.007s, peaked 0.000 MiB above current, total RAM usage 138.453 MiB


In [24]:
%timeit ca.sum()

The slowest run took 5.60 times longer than the fastest. This could mean that an intermediate result is being cached 
100 loops, best of 3: 2.07 ms per loop
In [24] used 0.012 MiB RAM in 1.084s, peaked 0.000 MiB above current, total RAM usage 138.465 MiB


This is around 4x slower than a regular NumPy array, but the size of the array is just a 20x smaller.

In order to see if this is because of the compression overhead, let's use an uncompressed array:

In [25]:
cau = bcolz.carray(a, cparams=bcolz.cparams(clevel=0))

In [25] used 8.152 MiB RAM in 0.103s, peaked 0.000 MiB above current, total RAM usage 146.617 MiB


In [26]:
cau2 = bcolz.carray(a, cparams=bcolz.cparams(clevel=0))

In [26] used 7.477 MiB RAM in 0.091s, peaked 0.000 MiB above current, total RAM usage 154.094 MiB


In [27]:
cau

carray((1000000,), int64)
  nbytes: 7.63 MB; cbytes: 7.75 MB; ratio: 0.98
  cparams := cparams(clevel=0, shuffle=True, cname='blosclz')
[     0      1      2 ..., 999997 999998 999999]

In [27] used 0.000 MiB RAM in 0.006s, peaked 0.000 MiB above current, total RAM usage 154.094 MiB


In [28]:
%timeit ca.sum()

100 loops, best of 3: 2.43 ms per loop
In [28] used 0.266 MiB RAM in 1.233s, peaked 0.000 MiB above current, total RAM usage 154.359 MiB


As we can see, the times with an uncompressed `carray` are very close to a compressed one.  On the other hand, carray is not competitive with NumPy, at least for this kind of operations. 